<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/%EC%98%81%EC%96%91%EC%A0%9C%EA%B3%B5%EC%BD%94%EB%93%9C%EB%B6%84%EC%84%9D%EC%A4%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"Isolation Forest" 모델 사용
001 feature 골라서 사용하면서 + pca 사용하여 최고점수 도출시도

002 코사인, 피어슨 유사도 특성추가

003 -

004 Isolation Forest가 Robust모델에 비해 f1, auc 스코어 간의 격차가 적어 hyperparameter값 세부조정 시도

- Isolation Forest h-params는 nes=150근처 pca는 사용하지 않을 때 성능이 가장 좋았다. -> 최대한 독립적인 특성만 사용하면서 학습시켰기 때문이라고 생각

005 성능 개선을 위해서는 특성추가가 필요하다고 생각 11, 12, 17에서
2, 4, 9, 10, 11, 12, 16, 17, 18 으로 특성 늘려서 학습



In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from itertools import permutations,combinations #특성의 조합을 구하기 위한 import

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")


import math #차후 factorial 계산 등 수학적인 계산을 위한 import
from numpy import dot #코사인 유사도 및 피어슨 유사도 구할 때 numpy 계산을 위한 import
from numpy.linalg import norm #위와 동일

from scipy.stats import pearsonr
from sklearn.preprocessing import PolynomialFeatures #특성공학을 사용하기 위한 import
from sklearn.preprocessing import StandardScaler #전처리된 데이터들의 scale을 맞춰주기 위한 import
from sklearn.decomposition import PCA #차원 축소 - 주성분의 개수에 따른 최적화 모델 구현을 위한 import



#model
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
# from sklearn.cluster import AgglomerativeClustering


from sklearn.metrics import roc_auc_score#점수산출용
from sklearn.metrics import f1_score#점수산출용
from sklearn.metrics import classification_report#점수산출용
from sklearn.metrics import accuracy_score #이후 test.csv 파일의 predict 자료의 score을 매기기 위한 import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train dataset은 Label이 존재하지 않음
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project_cfr/train.csv")
val_df  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project_cfr/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project_cfr/test.csv")
submit = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project_cfr/sample_submission.csv")

단, Test Dataset이 포함된 데이터로 사전 학습된 모델 사용 금지

대회 제공 데이터 이외의 외부데이터 사용 금지

대회 제공 데이터 중 Train Dataset (train.csv)만 모델 학습에 사용 가능

대회 제공 데이터 중 Validation Dataset (validation.csv)은 모델 학습에 사용은 불가능하지만 데이터 통계 정보와 모델 평가에는 사용 가능

대회 제공 데이터 중 Test Dataset (test.csv)은 리더보드 제출을 위한 추론에만 사용 가능 (모델 학습 사용 불가능)

In [ ]:
print(train_df.head(1))
print('--------------------------------------')
print(test_df.head(1))
print('--------------------------------------')
print(val_df.head(1))
print('--------------------------------------')
print(submit.head(1))

   ID        V1        V2        V3       V4        V5        V6        V7  \
0   3 -1.358354 -1.340163  1.773209  0.37978 -0.503198  1.800499  0.791461   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   

        V26       V27       V28       V29       V30  
0 -0.139097 -0.055353 -0.059752  4.983721 -0.994972  

[1 rows x 31 columns]
--------------------------------------
        ID        V1        V2        V3        V4        V5        V6  \
0  AAAA0x1 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388   

         V7        V8        V9  ...       V21       V22       V23       V24  \
0  0.239599  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928   

        V25       V26       V27       V28       V29       V30  
0  0.128539 -0.189115  0.133558 -0.021053  1.783274 -0.994983  

[1 rows x 31 columns]
--------------------------------------
   ID       

In [ ]:
clist = list(train_df.columns)
print(clist)

train_data = train_df[clist[1:]].to_numpy()
val_data = val_df[clist[1:]].to_numpy()
val_target = val_df['Class'].to_numpy()

print(val_target)

['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30']
[0 0 0 ... 0 0 0]


In [ ]:

valcount1=0
valcountm1=0
for i in val_target:
  if i==1:
    valcount1+=1
  else:
    valcountm1+=1

print(valcount1, valcountm1)

outliers_fraction = valcount1/(valcount1+valcountm1)
print("outliers_fraction =",outliers_fraction)

30 28432
outliers_fraction = 0.0010540369615627855


코사인유사도 구하기

In [ ]:
# 검증데이터 중 사기인 것들 안에서 V1-V30 까지
val_abnor = val_df[val_df['Class']==1][clist[1:]]
print(val_abnor[:2])
print("-----------------------")

#
abnor_vector = val_abnor.mean().to_numpy()
print(abnor_vector)

           V1        V2        V3        V4        V5        V6        V7  \
71  -3.043541 -3.157307  1.088463  2.288644  1.359805 -1.064823  0.325574   
641  0.316459  3.809076 -5.615159  6.047445  1.554026 -2.651353 -0.746579   

           V8        V9       V10  ...       V21       V22       V23  \
71  -0.067794 -0.270953 -0.838587  ...  0.661696  0.435477  1.375966   
641  0.055586 -2.678679 -4.959493  ...  0.208828 -0.511747 -0.583813   

          V24       V25       V26       V27       V28       V29       V30  
71  -0.293803  0.279798 -0.145362 -0.252773  0.035764  7.084469 -0.989438  
641 -0.219845  1.474753  0.491192  0.518868  0.402528 -0.293440 -0.906272  

[2 rows x 30 columns]
-----------------------
[-6.4902307   4.78656835 -9.80242181  5.45855983 -4.61786208 -1.83473187
 -7.69930106  1.31215119 -3.31825355 -7.57510465  4.45015242 -7.29492844
  0.05961486 -7.81510871 -0.26495949 -4.50606623 -7.33768492 -2.45956389
  0.60334874  0.35844264 -0.02350577  0.37863818  0.09228

In [ ]:
def cos_sim(a, b):
  return dot(a, b)/(norm(a)*norm(b))

def cosfeature(data):
  datacs=[]
  for i in data[clist[1:]].to_numpy():
    datacs.append(cos_sim(abnor_vector, i))
  data['V31']=datacs

cosfeature(train_df)
cosfeature(val_df)
cosfeature(test_df)

피어슨유사도 구하기 - 찾아보귕^^

In [ ]:
def pearson_similarity(a, b):
    return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

def pearsonfeature(data):
  datacs=[]
  for i in data[clist[1:]].to_numpy():
    datacs.append(pearson_similarity(abnor_vector, i))
  data['V32']=datacs

pearsonfeature(train_df)
pearsonfeature(val_df)
pearsonfeature(test_df)

clist = list(train_df.columns)

In [ ]:
print(len(clist))
print(clist)

33
['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32']


In [ ]:
col=[clist[i] for i in [11, 12, 17, 31, 32]]
print(col)

['V11', 'V12', 'V17', 'V31', 'V32']


In [ ]:
# ID 를 제외한 특성 불러오기 함수
def predata(data):
  return data.drop(columns=['ID'])[col] 


def get_pred_label(model_pred): # val_pred 이 들어간다면??  # val_pred = m.predict(val_pca) 즉, "model_rc로 차원축소한 val 데이터"를 넣는다면? # val_pred 는 차원축소한 val데이터를 rc모델로 예측한 값들이 들어가 있다.
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)  # np.where() 은 조건을 만족하는 위치의 인덱스를 가져옴
    model_pred = np.where(model_pred == -1, 1, model_pred) # np.where(a조건,b,c) a조건이면 b, 그렇지 않으면 c의 인덱스를 가져옴 
    return model_pred


train_x = predata(train_df) # train_df['V11','V12','V17','V31','V32']
print(train_x[:5])
print("-----------------")

val_x = predata(val_df) # Input Data
val_y = val_df['Class'] # Label
print(val_x[:1])
print(val_y[:1])

test_x = predata(test_df)
print(test_x[:1])
print(col)



        V11       V12       V17       V31       V32
0  0.624501  0.066084  1.109969 -0.026012  0.024835
1 -0.226487  0.178228 -0.684093 -0.090965 -0.136051
2  1.341262  0.359894 -0.058133 -0.045050  0.001014
3 -0.619468  0.291474 -1.222127 -0.108131 -0.144766
4 -0.705117 -0.110452 -0.499768 -0.059377  0.015230
-----------------
        V11      V12      V17       V31       V32
0  1.017614  0.83639 -0.54098 -0.055663  0.020418
0    0
Name: Class, dtype: int64
      V11       V12       V17       V31       V32
0 -0.5516 -0.617801  0.207971 -0.052446  0.015425
['V11', 'V12', 'V17', 'V31', 'V32']


In [ ]:
# rc모델만 학습

# for i in range(2, len())
scoreboard={}
th_score=0.88 #제출 파일 저장 문턱점수 
maxscore=0.88 #지난회차 최고점수
testname="002"
poly_dim =1

for i in range(4, len(col)+1):  # range(4,6) i = 4 , 5
  # 스케일 학습 후 적용
  ss = StandardScaler()
  ss.fit(train_x)
  train_x = ss.transform(train_x)
  val_x = ss.transform(val_x)
  test_x = ss.transform(test_x)
  # PCA 학습 후 적용 
  pca = PCA(n_components = i)
  pca.fit(train_x)
  train_pca = pca.transform(train_x)
  val_pca = pca.transform(val_x)
  test_pca = pca.transform(test_x)

  # anomaly_algorithms = [("Robust covariance", EllipticEnvelope(contamination=outliers_fraction)),
  #                     ("One-Class SVM", svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1)),
  #                     ("Isolation Forest", IsolationForest(max_samples=len(train_pca), contamination=outliers_fraction,random_state=42)),
  #                     ("Local Outlier Factor", LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction))]

  contamination_controlnumber=0
  
  # model_if = IsolationForest(n_estimators=nes, max_samples=len(train_pca), contamination=outliers_fraction, random_state=42, warm_start=True, verbose=0)
  # model_ocs = svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=10)
  model_rc = EllipticEnvelope(contamination=outliers_fraction + contamination_controlnumber) # 가우스 분산 데이터 세트에서 특이 치를 탐지하기위한 객체
  # model_lof = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)
  # ("One-Class SVM",model_ocs) ("Isolation Forest", model_if),
  model_list=[ ("Robust covariance", model_rc) ]
  for name, m in model_list: # m 은 model_re 모델
    m.fit(train_pca)

    val_pred = m.predict(val_pca) # model prediction # val_pred 는 차원축소한 val데이터를 rc모델로 예측한 값들이 들어가 있다.
    val_pred = get_pred_label(val_pred) # 위 정의된 함수로 돌리면 이상치들은 사기(1) 정상(0)으로 분류가 진행됨
    print("1번",val_pred) ##

    val_score = f1_score(val_y, val_pred, average='macro') # (val 타겟 값, 모델로 예측해서 라벨링된 타겟값, average='macro'?)
    val_auc_score = roc_auc_score(val_y, val_pred)
    scoreboard["model:{}, pca-ncom:{}, contamination:{}".format(name, i, outliers_fraction + contamination_controlnumber)]=\ # outliers_fraction + contamination_controlnumber ?? 이게뭐지
    "f1-score : {}, auc-score : {}".format(round(val_score, 4)*100, round(val_auc_score, 4)*100)

    if val_score>maxscore:
      print("!!max score!!")

    if name == "Isolation Forest":
      print('model:{}, pca-ncom:{}, n-esti:{}, contamination:{}'.format(name, i, nes, outliers_fraction + contamination_controlnumber),'\n',
          'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
      
    else:
      print('model:{}, pca-ncom:{}, contamination:{}'.format(name, i, outliers_fraction + contamination_controlnumber),'\n',
          'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
      
      
    # print(f'Validation F1 Score : [{val_score}]')
    # print(classification_report(val_y, val_pred))
    print()

# 문턱점수 넘으면 자동으로 저장
  if val_score>th_score:
    test_pred = m.predict(test_pca) # model prediction  
    test_pred = get_pred_label(test_pred)
    submit['Class'] = test_pred
    submit.to_csv('./submit{}_model_{}_ncom{}.csv'.format(testname, name, i), index=False)



[0 0 0 ... 0 0 0]
!!max score!!
model:Robust covariance, pca-ncom:4, contamination:0.0010540369615627855 
 f1-score : 89.28,		auc-score : 86.66

[0 0 0 ... 0 0 0]
!!max score!!
model:Robust covariance, pca-ncom:5, contamination:0.0010540369615627855 
 f1-score : 88.59,		auc-score : 86.66



def predata(data):
  return data.drop(columns=['ID'])[col]

def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

scoreboard={} #모델 별 점수 값 저장을 위한 딕셔너리 저장공간
maxscoreboard=[] #제일 좋았던 모델 별 점수 젖장을 위한 저장공간
th_score=0.88 #제출 파일 저장 문턱점수 
maxscore=0 #지난회차 최고점수
testname="003"
poly_dim =1
testnum = 0

for k in range(1, 6):
  for ctu in list(combinations([4,9,10,16,18], k)): #k값에 따라서 몇개 뽑을지 결정
     
    col = [clist[i] for i in [11,12,17,31,32]+list(ctu)] # col 내가 뽑은 특성들이 들어 있는 list
    
    train_x = predata(train_df)
    val_x = predata(val_df) # Input Data
    val_y = val_df['Class'] # Label
    test_x = predata(test_df)  

    for i in range(len(col)-2, len(col)+1): #특성 개수 => len(col)
      ss = StandardScaler()
      ss.fit(train_x)
      train_x = ss.transform(train_x)
      val_x = ss.transform(val_x)
      test_x = ss.transform(test_x)

      pca = PCA(n_components = i) #i값에 따라 pca 주성분 개수 결정
      pca.fit(train_x)
      train_pca = pca.transform(train_x)
      val_pca = pca.transform(val_x)
      test_pca = pca.transform(test_x)

      contamination_controlnumber=0 #확률 조정 변수
      nes=150 # Isolation Forest n_esti 값 조정하기 위한 변수


      model_if = IsolationForest(n_estimators=nes, max_samples=len(train_pca), contamination=outliers_fraction, random_state=42, verbose=0)
      # model_ocs = svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=10)
      model_rc = EllipticEnvelope(contamination=outliers_fraction + contamination_controlnumber)
      # model_lof = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)
      # ("One-Class SVM",model_ocs) 
      model_list=[("Isolation Forest", model_if), ("Robust covariance", model_rc) ]
      for name, m in model_list:
        testnum+=1
        m.fit(train_pca)

        val_pred = m.predict(val_pca) # model prediction
        val_pred = get_pred_label(val_pred)
        val_score = f1_score(val_y, val_pred, average='macro')
        val_auc_score = roc_auc_score(val_y, val_pred)
        scoreboard["model:{}, pca-ncom:{}, contamination:{}".format(name, i, outliers_fraction + contamination_controlnumber)]=\
        "f1-score : {}, auc-score : {}".format(round(val_score, 4)*100, round(val_auc_score, 4)*100)

        if val_score >= maxscore:
          maxscore = val_score
          maxscoreboard.append("no.{}-f1-score:{},\t\t auc-score:{}".format(testnum, val_score, val_auc_score))

          print("----------------------------------------------------")
          print("!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!")
          print("----------------------------------------------------")
        
        print("no.{} model_{} ------------ feature:{}, k:{}".format(testnum, name, col, k))
        
        if name == "Isolation Forest":
          print('pca-ncom:{}, n-esti:{}, contamination:{}'.format(i, nes, outliers_fraction + contamination_controlnumber),'\n',
              'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
        else:
          print('pca-ncom:{}, contamination:{}'.format(i, outliers_fraction + contamination_controlnumber),'\n',
              'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
          
        # print(f'Validation F1 Score : [{val_score}]')
        # print(classification_report(val_y, val_pred))
        print()

      if val_score>th_score:
        test_pred = m.predict(test_pca) # model prediction  
        test_pred = get_pred_label(test_pred)
        submit['Class'] = test_pred
        submit.to_csv('./submit{}_no.{}_model_{}.csv'.format(testname, testnum, name), index=False)



In [ ]:
list(combinations([4,9,10,16,18], 3))